In [ ]:
import os, shutil
from os import listdir
from os.path import isfile, join
import random

In [ ]:
print(os.getcwd())

#creo lista di nomi 
directories = ['data/train/', 
               'data/val/', 
               'data/test/']

train_file_names = [f for f in listdir(directories[0]) if isfile(join(directories[0], f))]
val_file_names = [f for f in listdir(directories[1]) if isfile(join(directories[1], f))]
test_file_names = [f for f in listdir(directories[2]) if isfile(join(directories[2], f))]

_file_names = [train_file_names, val_file_names, test_file_names] 

In [ ]:
#numero immagini per categoria
scans = ['CT','MRI','PET']
numbers=[]
minimi=[]

for directory in directories:    
    for scan in scans:
        a = len([f for f in listdir(directory) if f[:2]==scan[:2]])
        #print('Numbero di immagini', scan, 'in', directory,':', a)
        numbers.append(a)
    minimi.append(min(numbers))

#print(minimi)
#=> è sempre la PET che ha il numero minore

In [ ]:
#creo la lista di file bilanciata: n.b  la percentuale viene mantenuta
train_final_file_names=[]
val_final_file_names=[]
test_final_file_names=[]
_final_file_names = [train_final_file_names, val_final_file_names, test_final_file_names] 

l=[]
for directory,minimo,name in zip(directories, minimi,_final_file_names):    
    for scan in scans:
        l = [f for f in listdir(directory) if f[:2]==scan[:2]]
        random.shuffle(l)
        l = l[:minimo]
        name.extend(l) #estendo la lista (don't append)
        
# How I cicled ^^^

#    minimo |
#    name   |
#    train  | val | test
# 
# CT    ... | ... | ...
# MRI   ... | ... | ...
# PET   ... | ... | ...

In [ ]:
print("DATASET")
print("Training", len(train_final_file_names), ", Validation:", len(val_final_file_names) ,", Test:", len(test_final_file_names))

## Converto le immagini in tensori

In [ ]:
from skimage import io
import matplotlib.pyplot as plt
import cv2

### X

In [ ]:
def WriteAndArrayImages(path):
    x = io.imread(path[random.randint(0,10)])
    x = cv2.resize(x, dsize=(128,128))
    
    for img in path[1:]:
        img = io.imread(img)
        img = cv2.resize(img, dsize=(128,128))
        x = np.dstack((x,img))
    x = np.rollaxis(x,-1)
    return x

In [ ]:
import numpy as np

#paths per leggere le immagini
train_paths=[]
val_paths=[]
test_paths=[]
_paths=[train_paths,val_paths,test_paths]

for path,name,directory in zip(_paths,_final_file_names,directories):
    for f in name:
        path.append(directory+f)

#leggo le immagini e le stacco una sopra l'altra
x_train = WriteAndArrayImages(train_paths)
x_val = WriteAndArrayImages(val_paths)
x_test = WriteAndArrayImages(test_paths)

print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

#plot images from each dataset
f, axarr = plt.subplots(1,3,figsize=(12, 12))
axarr[0].imshow(x_train[random.randint(0,100)], cmap=plt.cm.gray)
axarr[1].imshow(x_val[random.randint(0,100)], cmap=plt.cm.gray)
axarr[2].imshow(x_test[random.randint(0,100)], cmap=plt.cm.gray)

### Y

In [ ]:
def MakeLabels(_name_files):
    y=[]
    for text in _name_files:
        fileNameOnly = text[:text.find(ext)]
        y.append(''.join([i for i in fileNameOnly if not i.isdigit()]))
    for i, item in enumerate(y):
        if item == 'CT':
            y[i] = 0
        elif item == 'MRI':
            y[i] = 1
        else: y[i] = 2    
    y = np.concatenate((y,))
    return y

In [ ]:
#estensione immagini
ext = '.png'

#costruisco le lables
y_train = MakeLabels(_final_file_names[0])
y_val = MakeLabels(_final_file_names[1])
y_test = MakeLabels(_final_file_names[2])

In [ ]:
#check
X = [x_train, x_val, x_test]
Y = [y_train, y_val, y_test]

for x,y in zip(X,Y):
    if len(x) != len(y):
        print(len(x))
        print(len(y))
        print('Error! La lunghezza delle liste non combacia!')
    else: 
        print('Ok, le lunghezze combaciano con le X')

#controlla numero di cluster (expected 3)
n_clusters = len(np.unique(y))
print('Clusters:', n_clusters)

## Autoencoder Model 
N.B. Execute only the model you want to train 

In [ ]:
from keras.models import Model
from keras import backend as K
from keras import layers
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape, Conv2DTranspose
from keras.models import Model

### Convolutional Autoencoder Model

In [ ]:
def autoencoderConv2D_1(input_shape=(128, 128, 1), filters=[32, 64, 128, 256, 3]):
    input_img = Input(shape=input_shape)
    if input_shape[0] % 8 == 0:
        pad3 = 'same'
    else:
        pad3 = 'valid'
    x = Conv2D(filters[0], 5, strides=2, padding='same', activation='relu', name='conv1', input_shape=input_shape)(input_img)

    x = Conv2D(filters[1], 5, strides=2, padding='same', activation='relu', name='conv2')(x)

    x = Conv2D(filters[2], 3, strides=2, padding='same', activation='relu', name='conv3')(x)
    
    x = Flatten()(x)
    
    encoded = Dense(units=filters[3], name='embedding')(x)
    
    y = Dense(units=filters[4], name='output_to_cluster')(encoded)
    
    x = Dense(units=filters[2]*int(input_shape[0]/8)*int(input_shape[0]/8), activation='relu')(encoded)

    x = Reshape((int(input_shape[0]/8), int(input_shape[0]/8), filters[2]))(x)
    
    x = Conv2DTranspose(filters[1], 3, strides=2, padding='same', activation='relu', name='deconv3')(x)

    x = Conv2DTranspose(filters[0], 5, strides=2, padding='same', activation='relu', name='deconv2')(x)

    decoded = Conv2DTranspose(input_shape[2], 5, strides=2, padding='same', name='deconv1')(x)
    
    return Model(inputs=input_img, outputs=decoded, name='AE'), Model(inputs=input_img, outputs=y, name='encoder')

In [ ]:
autoencoder, encoder = autoencoderConv2D_1()

print('ENCODER')
encoder.summary()

print('AUTOENCODER')
autoencoder.summary()

### Reshape Arrays for AE

In [ ]:
#use this function for reshaping arrays for CAE
def ReshapeCAE(array):
    array = array.reshape(array.shape + (1,))
    array = array/255.
    return array

x_train = ReshapeCAE(x_train)
x_val = ReshapeCAE(x_val)
x_test = ReshapeCAE(x_test)

print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

### Clustering Layer

In [ ]:
from keras.engine.topology import Layer

class ClusteringLayer(Layer):
    """
    Clustering layer converts input sample (feature) to soft label, i.e. a vector that represents the probability of the
    sample belonging to each cluster. The probability is calculated with student's t-distribution.

    # Example
    ```
        model.add(ClusteringLayer(n_clusters=10))
    ```
    # Arguments
        n_clusters: number of clusters.
        weights: list of Numpy array with shape `(n_clusters, n_features)` witch represents the initial cluster centers.
        alpha: degrees of freedom parameter in Student's t-distribution. Default to 1.0.
    # Input shape
        2D tensor with shape: `(n_samples, n_features)`.
    # Output shape
        2D tensor with shape: `(n_samples, n_clusters)`.
    """

    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights
        self.input_spec = InputSpec(ndim=2)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight(shape=(self.n_clusters, input_dim), initializer='glorot_uniform', name='clusters')
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        """ student t-distribution, as same as used in t-SNE algorithm.
         Measure the similarity between embedded point z_i and centroid µ_j.
                 q_ij = 1/(1+dist(x_i, µ_j)^2), then normalize it.
                 q_ij can be interpreted as the probability of assigning sample i to cluster j.
                 (i.e., a soft assignment)
        Arguments:
            inputs: the variable containing data, shape=(n_samples, n_features)
        Return:
            q: student's t-distribution, or soft labels for each sample. shape=(n_samples, n_clusters)
        """
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1)) # Make sure each sample's 10 values add up to 1.
        return q

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters}
        base_config = super(ClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

## Pretrain

### Hyperparameters (to edit)

In [ ]:
from keras.initializers import VarianceScaling
from keras.optimizers import SGD

init = VarianceScaling(scale=1. / 3., mode='fan_in', distribution='uniform')

pretrain_epochs = 50 #####
batch_size = 16 #####

In [ ]:
#dir results
save_dir = './results'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [ ]:
from keras.callbacks import TensorBoard

autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.fit(x_train, 
                x_train, 
                batch_size=batch_size, 
                epochs=pretrain_epochs, 
                validation_data=(x_val, x_val),
                #callbacks=[TensorBoard(log_dir='/tmp/autoencoder')]
               )

autoencoder.save_weights(save_dir+'/conv_ae_weights.h5')

In [ ]:
autoencoder.load_weights(save_dir+'/conv_ae_weights.h5')

tensorboard --logdir=/tmp/autoencoder

### Plot train and validation loss 

In [ ]:
#plot train and validation loss of pretraining
plt.plot(autoencoder.history.history['loss'])
plt.plot(autoencoder.history.history['val_loss'])
plt.title('AE loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

### Show prediction after pretraining CAE

In [ ]:
#predict images and display

directories = ['data/train/', 
               'data/val/', 
               'data/test/']

scans=['CT','MRI','PET']

def PickListImagesPerType(directory,scan):
    names = [f for f in listdir(directory) if f[:2]==scan[:2]]
    return names

def PickOneImage(names,directory,n):
    image = io.imread(directory+names[n])
    image = cv2.resize(image, dsize=(128,128))
    return image

In [ ]:
#image layer to display (change this to display different images and respective decoded ones)
n = 5

### Prediction on test dataset

In [ ]:
#ORIGINAL IMAGES
#pick one image per type of scan
imgsCT = PickOneImage(PickListImagesPerType(directories[0],scans[0]),directories[0],n)
imgsMRI = PickOneImage(PickListImagesPerType(directories[0],scans[1]),directories[0],n)
imgsPET = PickOneImage(PickListImagesPerType(directories[0],scans[2]),directories[0],n)

#DECODED IMAGES
#costruisco la lista di immagini da decodificare
listaCT=[]
for f in PickListImagesPerType(directories[0],scans[0]): 
    listaCT.append(join(directories[0], f))

listaMRI=[]
for f in PickListImagesPerType(directories[0],scans[1]): 
    listaMRI.append(join(directories[0], f))
    
listaPET=[]
for f in PickListImagesPerType(directories[0],scans[2]): 
    listaPET.append(join(directories[0], f))

#decode images per type of scan
decoded_imgsCT = autoencoder.predict(ReshapeCAE(WriteAndArrayImages(listaCT)))
decoded_imgsMRI = autoencoder.predict(ReshapeCAE(WriteAndArrayImages(listaMRI)))
decoded_imgsPET = autoencoder.predict(ReshapeCAE(WriteAndArrayImages(listaPET)))

#select one decoded image and make in plottable
decoded_imgCT = decoded_imgsCT[n].reshape(128,128)
decoded_imgMRI = decoded_imgsMRI[n].reshape(128,128)
decoded_imgPET = decoded_imgsPET[n].reshape(128,128)

In [ ]:
fig = plt.figure(figsize=(13,10))

#plot images
ax = plt.subplot(2,3,1)
plt.imshow(imgsCT)
plt.gray()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

ax = plt.subplot(2,3,2)
plt.imshow(imgsMRI)
plt.gray()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

ax = plt.subplot(2,3,3)
plt.imshow(imgsPET)
plt.gray()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

ax = plt.subplot(2,3,4)
plt.imshow(decoded_imgCT)
plt.gray()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

ax = plt.subplot(2,3,5)
plt.imshow(decoded_imgMRI)
plt.gray()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

ax = plt.subplot(2,3,6)
plt.imshow(decoded_imgPET)
plt.gray()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

plt.subplots_adjust(wspace=0.2, hspace=-0.3)

## DCEC Model

In [ ]:
from keras.engine.topology import InputSpec

clustering_layer = ClusteringLayer(n_clusters, name='clustering')(encoder.output)
model = Model(inputs=encoder.input, outputs=[clustering_layer, autoencoder.output])

In [ ]:
from keras.utils import plot_model
from IPython.display import Image

plot_model(model, to_file='DCEC_model.png', show_shapes=True)
Image(filename='DCEC_model.png')

## Inizialize cluster centers using k-means

In [ ]:
from sklearn.cluster import KMeans
import metrics as metrics

feature_model = Model(input=encoder.input, outputs=encoder.get_layer(name='output_to_cluster').output)
features = feature_model.predict(x_train)
km = KMeans(n_clusters=n_clusters)

features = np.reshape(features, newshape=(features.shape[0], -1))
y_pred = km.fit_predict(features)
print('acc=', metrics.acc(y_train, y_pred), 'nmi=', metrics.nmi(y_train, y_pred), 'ari=', metrics.ari(y_train, y_pred))

y_pred_last = np.copy(y_pred)

In [ ]:
model.compile(loss=['kld', 'mse'], loss_weights=[0.1, 1], optimizer='adam')

## Training

### Deep clustering

In [ ]:
def target_distribution(q):
    weight = q ** 2 / q.sum(0)
    return (weight.T / weight.sum(1)).T

#for plotting
lista_train_losses = [[],[],[]]
lista_val_losses = [[],[],[]]
iterazione=[]

train_loss = [0,0,0]
val_loss = [0,0,0]
index = 0
maxiter = 100
update_interval = 1
train_index_array = np.arange(x_train.shape[0])
val_index_array = np.arange(x_val.shape[0])

tol = 0.001 # tolerance threshold to stop training

### Start training

In [ ]:
import metrics
from IPython.core.debugger import set_trace
#set_trace()


for ite in range(int(maxiter)):
    if ite % update_interval == 0:
        q, _ = model.predict(x_train, verbose=0)
        p = target_distribution(q)  # update the auxiliary target distribution p

        # evaluate the clustering performance
        y_pred = q.argmax(1)
        if y_train is not None:
            acc = np.round(metrics.acc(y_train, y_pred), 5)
            nmi = np.round(metrics.nmi(y_train, y_pred), 5)
            ari = np.round(metrics.ari(y_train, y_pred), 5)
            train_loss = np.round(train_loss, 5)
            val_loss = np.round(val_loss, 5)
            print('Iter', ite, ': Acc', acc, ', nmi', nmi, ', ari', ari, '; train_loss=', train_loss, ';val_loss=', val_loss) 

        # check stop criterion
        delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
        y_pred_last = np.copy(y_pred)
        if ite > 0 and delta_label < tol:
            print('delta_label ', delta_label, '< tol ', tol)
            print('Reached tolerance threshold. Stopping training.')
            break
    
    # train on batch
    idx_train = train_index_array[index * batch_size: min((index+1) * batch_size, x_train.shape[0])]
    train_loss = model.train_on_batch(x=x_train[idx_train], y=[p[idx_train], x_train[idx_train]])
    lista_train_losses[0].append(train_loss[0])
    lista_train_losses[1].append(train_loss[1])
    lista_train_losses[2].append(train_loss[2])
    
    idx_val = val_index_array[index * batch_size: min((index+1) * batch_size, x_val.shape[0])]
    val_loss = model.test_on_batch(x=x_val[idx_val], y=[p[idx_val], x_val[idx_val]])
    lista_val_losses[0].append(val_loss[0])
    lista_val_losses[1].append(val_loss[1])
    lista_val_losses[2].append(val_loss[2])
    
    index = index + 1 if (index + 1) * batch_size <= x_train.shape[0] else 0
    
    iterazione.append(ite)
    

# save the trained model
print('saving model to:', save_dir + '/dcec_model_final.h5')
model.save_weights(save_dir + '/dcec_model_final.h5')

### Plot DCEC train and validation losses

In [ ]:
#plot
plt.subplot(1,3,1)
x1=iterazione
y1=lista_train_losses[0]
y2=lista_val_losses[0]
plt.plot(x1,y1)
plt.plot(x1,y2)
plt.title('L')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])

plt.subplot(1,3,2)
x1=iterazione
y1=lista_train_losses[1]
y2=lista_val_losses[1]
plt.plot(x1,y1)
plt.plot(x1,y2)
plt.title('Lr')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])

plt.subplot(1,3,3)
x1=iterazione
y1=lista_train_losses[2]
y2=lista_val_losses[2]
plt.plot(x1,y1)
plt.plot(x1,y2)
plt.title('Lc')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])

plt.show()

# Final evaluation on test dataset

In [ ]:
q, _ = model.predict(x_test, verbose=0)
p = target_distribution(q) 

test_loss = model.fit(x=x_test, y=[p, x_test])
print(' Acc', acc) 

## Show prediction after pretraining DCEC

### Prediction on test dataset

In [ ]:
#image layer to display (change this to display different images and respective decoded ones)
n = 5 

#ORIGINAL IMAGES
#pick one image per type of scan
imgsCT = PickOneImage(PickListImagesPerType(directories[2],scans[0]),directories[2],n)
imgsMRI = PickOneImage(PickListImagesPerType(directories[2],scans[1]),directories[2],n)
imgsPET = PickOneImage(PickListImagesPerType(directories[2],scans[2]),directories[2],n)

#DECODED IMAGES
#costruisco la lista di immagini da decodificare
listaCT=[]
for f in PickListImagesPerType(directories[2],scans[0]): 
    listaCT.append(join(directories[2], f))

listaMRI=[]
for f in PickListImagesPerType(directories[2],scans[1]): 
    listaMRI.append(join(directories[2], f))
    
listaPET=[]
for f in PickListImagesPerType(directories[2],scans[2]): 
    listaPET.append(join(directories[2], f))

#decode images per type of scan
decoded_imgsCT = model.predict(ReshapeCAE(WriteAndArrayImages(listaCT)))[1]
decoded_imgsMRI = model.predict(ReshapeCAE(WriteAndArrayImages(listaMRI)))[1]
decoded_imgsPET = model.predict(ReshapeCAE(WriteAndArrayImages(listaPET)))[1]

#select one decoded image and make in plottable
decoded_imgCT = decoded_imgsCT[n].reshape(128,128)
decoded_imgMRI = decoded_imgsMRI[n].reshape(128,128)
decoded_imgPET = decoded_imgsPET[n].reshape(128,128)

#plot images
ax = plt.subplot(2,3,1)
plt.imshow(imgsCT)
plt.gray()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

ax = plt.subplot(2,3,2)
plt.imshow(imgsMRI)
plt.gray()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

ax = plt.subplot(2,3,3)
plt.imshow(imgsPET)
plt.gray()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

ax = plt.subplot(2,3,4)
plt.imshow(decoded_imgCT)
plt.gray()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

ax = plt.subplot(2,3,5)
plt.imshow(decoded_imgMRI)
plt.gray()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

ax = plt.subplot(2,3,6)
plt.imshow(decoded_imgPET)
plt.gray()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

## Plotting clusters

### Train

In [ ]:
features = feature_model.predict(x_train)
y_pred = km.fit_predict(features)
plt.scatter(features[:,0], features[:,1], c=y_pred, s=50, cmap='brg')
centers = km.cluster_centers_
plt.scatter(centers[:,0], centers[:,1], c='black', s=200, alpha=0.5)

### Test

In [ ]:
features = feature_model.predict(x_test)
y_pred = km.fit_predict(features)
plt.scatter(features[:,0], features[:,1], c=y_pred, s=50, cmap='brg')
centers = km.cluster_centers_
plt.scatter(centers[:,0], centers[:,1], c='black', s=200, alpha=0.5)

## Confusion matrix

In [ ]:
import seaborn as sns
import sklearn.metrics
import matplotlib.pyplot as plt
sns.set(font_scale=3)
confusion_matrix = sklearn.metrics.confusion_matrix([int(i) for i in y_test], y_pred)

plt.figure(figsize=(16, 14))
sns.heatmap(confusion_matrix, annot=True, fmt="d", annot_kws={"size": 20});
plt.title("Confusion matrix")
plt.ylabel('True label')
plt.xlabel('Clustering label')
plt.show()

In [ ]:
from sklearn.utils.linear_assignment_ import linear_assignment

y_true = y.astype(np.int64)
D = max(y_pred.max(), y_true.max()) + 1
w = np.zeros((D, D), dtype=np.int64)
# Confusion matrix.
for i in range(y_pred.size):
    w[y_pred[i], y_true[i]] += 1
ind = linear_assignment(-w)

sum([w[i, j] for i, j in ind]) * 1.0 / y_pred.size

In [ ]:
ind